<a href="https://colab.research.google.com/github/alfinmaulani/skripsi/blob/master/3ekstraksi_dan_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade scikit-learn
import numpy as np
import pandas as pd
import io
import os
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


load korpus

In [ ]:
url1 = 'https://raw.githubusercontent.com/alfinmaulani/skripsi/master/dataisear(preputama).csv'
url2 = 'https://raw.githubusercontent.com/alfinmaulani/skripsi/master/dataisear(preptambah).csv'
df = pd.read_csv(url2)

In [ ]:
temp=[]
for senten in df.dokumen:
  temp.append(ast.literal_eval(senten))
df.dokumen=temp

pilih model

In [ ]:
pakai_model=1

load model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if pakai_model == 1:#load model word2vec
  from gensim.models import word2vec
  model = gensim.models.word2vec.Word2Vec.load('https://github.com/alfinmaulani/skripsi/blob/master/w2v(preputama).model?raw=true')
elif pakai_model == 2:#load model glove
  !pip install glove-python-binary
  from glove import Glove
  from glove import Corpus
  model=Glove.load('/content/drive/MyDrive/glove.model')

cek model word2vec

In [ ]:
w2v.wv.vectors.shape

100

In [ ]:
w2v.wv["moment"]

(9352, 100)

cek model glove

In [ ]:
glove.word_vectors.shape

array([[-1.38042110e-01, -7.22731061e-02,  8.60750999e-02, ...,
         2.08342204e-01, -1.09346167e-03,  2.49755065e-01],
       [-4.86669324e-02, -3.01531564e-02,  2.96433428e-02, ...,
         7.29205770e-02,  1.32343629e-03,  8.70889215e-02],
       [-2.10199762e-01, -7.48029439e-02,  1.29497352e-01, ...,
         2.79371546e-01, -3.23691685e-02,  3.27171828e-01],
       ...,
       [ 8.08056025e-03, -6.31030540e-04, -3.63506564e-03, ...,
        -1.06357003e-02,  8.07690243e-05, -1.00125495e-02],
       [ 1.02141356e-02,  2.92651032e-03, -7.15933501e-03, ...,
        -1.50295605e-02, -3.31608783e-03, -1.47873352e-02],
       [-2.46649635e-04, -3.15596556e-03, -5.07804675e-03, ...,
        -1.69652998e-03,  4.01892947e-03, -2.10104997e-03]])

In [ ]:
glove.word_vectors[glove.dictionary['moment']]

(9352, 100)

average base ekstraksi fitur

In [ ]:
if pakai_model == 1:
  def eksfit(sentence, model):
    vec = [model[word] for word in word_tokenize(sentence)]
    sent_vec = np.mean(vec, axis=0)
    return sent_vec
  has_vec=[eksfit(sentences,model) for sentences in df.dokumen]
  has_vec=np.array(has_vec)
  has_vec
elif pakai_model == 2:
  import array
  ekstrak=[]
  temp=array.array('i',(0,)*100)
  for senten in sentences:
    for word in senten:
      temp=np.add(temp,model.word_vectors[model.dictionary[word]])
    temp=temp/len(senten)
    ekstrak.append(temp)
  has_vec=ekstrak

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
X=has_vec
y=df.emosi
print(np.shape(X))
print(np.shape(y))
# print(X)
# print(y)

(7516, 100)
(7516,)


klasifikasi svm

In [ ]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
def klasifikasi(krnl):
  clf = svm.SVC(kernel=krnl, decision_function_shape='ovr')
  y_pred = cross_val_predict(clf, X, y, cv=10)
  ConfusionMatrixDisplay.from_predictions(y, y_pred)
  plt.show()
  print(accuracy_score(y, y_pred))
  print(precision_recall_fscore_support(y, y_pred, average='macro'))

0.3253107558716038

In [ ]:
klasifikasi('poly')

0.31918842252882673

In [ ]:
klasifikasi('rbf')

0.3165261920276511

In [ ]:
klasifikasi('linear')

0.20889027396095985